# Taxi v-3 

In [18]:
#Hacemos los imports
import numpy as np
import gym
import random

In [19]:
# Creamos el ambiente, un espacio de 5x5, donde el taxi spawnea aleatoriamente, el pasajero se va a ubicar
# en una de cuatro posiciones R, G, B o Y 
# y va a ir hacia una de esas posiciones también.

ambiente = gym.make("Taxi-v3")
ambiente.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [20]:
# Buscamos la cantidad de estados que va a tener el sistema (en este caso 500 = 25 * 4 * 5) tal que 25 es la cantidad
# de posiciones en las que puede estar el taxi, 5 la cantidad de lugares donde puede estar el taxi (R,G,B,Y o sobre el taxi) y 4 por
# la cantidad de lugares de destino.
espacio_de_estados = ambiente.observation_space.n

# Buscamos la cantidad de acciones posibles, en este caso será 6 (subir, bajar, izquierda, derecha, subir pasajero y bajar pasajero)
espacio_de_acciones = ambiente.action_space.n


#Comprobamos con prints
print("Hay ", espacio_de_estados, " estados posibles")
print("Hay ", espacio_de_acciones, " acciones posibles")

Hay  500  estados posibles
Hay  6  acciones posibles


In [21]:
# Creamos la tabla Q, y la inicializamos en 0:

tabla_Q = np.zeros((espacio_de_estados,espacio_de_acciones))
print(tabla_Q)
print(tabla_Q.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


In [22]:
# Definimos los hiperparámetros
cantidad_episodios_de_entrenamiento = 25000
cantidad_episodios_de_test = 5
maxima_cantidad_de_pasos = 200

tasa_de_aprendizaje = 0.01
gamma = 0.99                                    # Tasa de descuento

# Parámetros de exploración
epsilon = 1.0                                   # Tasa de exploración
max_epsilon = 1.0                               # Exploración al principio
min_epsilon = 0.001                             # Minima probabilidad de exploracion posible
decay_rate = 0.01                               # Tasa de decaimiento exponencial para la exploración



In [23]:
# Definimos la política epsilon-greedy (va a ser nuestra política de acción a realizar)

def politica_epsilon_greedy(tabla_Q, estado):
  # Si el valor aleatorio generado es mayor que épsilon -> Hago explotación
  if(random.uniform(0,1) > epsilon):
    action = np.argmax(tabla_Q[estado])
  # Si no -> Hago exploración
  else:
    action = ambiente.action_space.sample()

  return action

In [24]:
# Definimos el algoritmo de Q-learning y entrenamos al agente

for episodio in range(cantidad_episodios_de_entrenamiento):
  # Reseteamos el ambiente
  estado = ambiente.reset()
  step = 0
  terminado = False

  # Reducimos el épsilon (para tener cada vez menos exploracion)
  epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episodio)

  for step in range(maxima_cantidad_de_pasos):
    # Defino la accion a realizar
    accion = politica_epsilon_greedy(tabla_Q,estado)

    # Realizo la accion y analizo, la recompensa y el estado actual
    nuevo_estado, recompensa, terminado, info = ambiente.step(accion)

    # Actualizo la tabla Q
    tabla_Q[estado][accion] = tabla_Q[estado][accion] + tasa_de_aprendizaje * (recompensa + gamma * np.max(tabla_Q[nuevo_estado]) - tabla_Q[estado][accion])

    # Si terminé : finalizo el episodio
    if terminado == True:
      break

    # Actualizo el estado
    estado = nuevo_estado

In [29]:
# Terminado el entrenamiento, procedemos a ejecutar los tests
import time
recompensas = []

frames = []
for episodio in range(cantidad_episodios_de_test):
  estado = ambiente.reset()
  step = 0
  terminado = False
  recompensa_total = 0
  print("****************************************************")
  print("EPISODIO ", episodio)

  for step in range(maxima_cantidad_de_pasos):
    ambiente.render()
    time.sleep(1)

    accion = politica_epsilon_greedy(tabla_Q,estado)
    nuevo_estado, recompensa, terminado, info = ambiente.step(accion)
    recompensa_total += recompensa

    if terminado:
      recompensas.append(recompensa_total)
      break
    
    estado = nuevo_estado
    
ambiente.close()
print("Recompensas obtenidas: " + str(sum(recompensas)/cantidad_episodios_de_test))

****************************************************
EPISODIO  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+-----